# Import Library

All credit due to AmbrosM, this notebook is useful that I'm using it for teaching my apprentices ^^

In [ ]:
import pandas as pd
import numpy as np
import pickle
import itertools
import gc
import math
import matplotlib.pyplot as plt
import dateutil.easter as easter
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
from datetime import datetime, date, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, HuberRegressor

# Mengambil Data

In [ ]:
original_train_df = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
original_test_df = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
gdp_df = pd.read_csv('../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')

gdp_df.set_index('year', inplace=True)

# The dates are read as strings and must be converted
for df in [original_train_df, original_test_df]:
    df['date'] = pd.to_datetime(df.date)
    df.set_index('date', inplace=True, drop=False)
original_train_df.head(2)

## Rumus SMAPE (Rumus untuk menghitung kesalahan prediksi)

In [ ]:
def smape_loss(y_true, y_pred):
    """SMAPE Loss"""
    return np.abs(y_true - y_pred) / (y_true + np.abs(y_pred)) * 200

#print(smape_loss(tf.constant([1, 2]), tf.constant([3, 4]))) # should print [100, 66.6667]

# Simple feature engineering

Dari data yang ada, akan ditambahkan hal - hal berikut pada data nya :
- Negara, Toko, Produk
- Hari Kerja
- Variasi musiman tiap produk sebagai seri Fourier dengan frekuensi dari 1 tahun hingga 18 hari
- GDP per negara

Residu dari model yang simpel ini akan mempermudah memahami hari libur

In [ ]:
# Feature engineering
def engineer(df):
    """Return a new dataframe with the engineered features"""
    
    def get_gdp(row):
        country = 'GDP_' + row.country
        return gdp_df.loc[row.date.year, country]
        
    new_df = pd.DataFrame({'gdp': np.log(df.apply(get_gdp, axis=1)),
                           'wd4': df.date.dt.weekday == 4, # Friday
                           'wd56': df.date.dt.weekday >= 5, # Saturday and Sunday
                          })

    # One-hot encoding (no need to encode the last categories)
    for country in ['Finland', 'Norway']:
        new_df[country] = df.country == country
    new_df['KaggleRama'] = df.store == 'KaggleRama'
    for product in ['Kaggle Mug', 'Kaggle Sticker']:
        new_df[product] = df['product'] == product
        
    # Seasonal variations (Fourier series)
    # The three products have different seasonal patterns
    dayofyear = df.date.dt.dayofyear
    for k in range(1, 20):
        new_df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
        new_df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
        new_df[f'mug_sin{k}'] = new_df[f'sin{k}'] * new_df['Kaggle Mug']
        new_df[f'mug_cos{k}'] = new_df[f'cos{k}'] * new_df['Kaggle Mug']
        new_df[f'sticker_sin{k}'] = new_df[f'sin{k}'] * new_df['Kaggle Sticker']
        new_df[f'sticker_cos{k}'] = new_df[f'cos{k}'] * new_df['Kaggle Sticker']

    return new_df

train_df = engineer(original_train_df)
train_df['date'] = original_train_df.date
train_df['num_sold'] = original_train_df.num_sold.astype(np.float32)
test_df = engineer(original_test_df)

features = test_df.columns

for df in [train_df, test_df]:
    df[features] = df[features].astype(np.float32)
print(list(features))

# Melatih model simpel (tanpa hari libur)

Model akan dilatih pada full data training. Validasi bisa nanti.

Pengecekan cepat, kita membuat plot prediksi nya `country='Norway', store='KaggleMart', product='Kaggle Hat'`.


In [ ]:
def fit_model(X_tr, X_va=None, outliers=False):
    """Scale the data, fit a model, plot the training history and validate the model"""
    start_time = datetime.now()

    # Preprocess the data
    X_tr_f = X_tr[features]
    preproc = StandardScaler()
    X_tr_f = preproc.fit_transform(X_tr_f)
    y_tr = X_tr.num_sold.values.reshape(-1, 1)
    
    # Train the model
    #model = LinearRegression() # 5.80558
    model = HuberRegressor(epsilon=1.20, max_iter=500, alpha = 7e-4) # 5.80143 (epsilon=1.20) *****************
    model.fit(X_tr_f, np.log(y_tr).ravel())

    if X_va is not None:
        # Preprocess the validation data
        X_va_f = X_va[features]
        X_va_f = preproc.transform(X_va_f)
        y_va = X_va.num_sold.values.reshape(-1, 1)

        # Inference for validation
        y_va_pred = np.exp(model.predict(X_va_f)).reshape(-1, 1)
        
        # Evaluation: Execution time and SMAPE
        smape_before_correction = np.mean(smape_loss(y_va, y_va_pred))
        #y_va_pred *= LOSS_CORRECTION
        smape = np.mean(smape_loss(y_va, y_va_pred))
        print(f"Fold {run}.{fold} | {str(datetime.now() - start_time)[-12:-7]}"
              f" | SMAPE: {smape:.5f}   (before correction: {smape_before_correction:.5f})")
        print(np.mean(smape_loss(y_va, y_va_pred)))
        
        # Plot y_true vs. y_pred
        plt.figure(figsize=(10, 10))
        plt.scatter(y_va, y_va_pred, s=1, color='r')
        #plt.scatter(np.log(y_va), np.log(y_va_pred), s=1, color='g')
        plt.plot([plt.xlim()[0], plt.xlim()[1]], [plt.xlim()[0], plt.xlim()[1]], '--', color='k')
        plt.gca().set_aspect('equal')
        plt.xlabel('y_true')
        plt.ylabel('y_pred')
        plt.title('OOF Predictions')
        plt.show()
        
    return preproc, model

preproc, model = fit_model(train_df)

# Plot all num_sold_true and num_sold_pred (five years) for one country-store-product combination
def plot_five_years_combination(engineer, country='Norway', store='KaggleMart', product='Kaggle Hat'):
    demo_df = pd.DataFrame({'row_id': 0,
                            'date': pd.date_range('2015-01-01', '2019-12-31', freq='D'),
                            'country': country,
                            'store': store,
                            'product': product})
    demo_df.set_index('date', inplace=True, drop=False)
    demo_df = engineer(demo_df)
    demo_df['num_sold'] = np.exp(model.predict(preproc.transform(demo_df[features])))
    plt.figure(figsize=(20, 6))
    plt.plot(np.arange(len(demo_df)), demo_df.num_sold, label='prediction')
    train_subset = train_df[(original_train_df.country == country) & (original_train_df.store == store) & (original_train_df['product'] == product)]
    plt.scatter(np.arange(len(train_subset)), train_subset.num_sold, label='true', alpha=0.5, color='red', s=3)
    plt.legend()
    plt.title('Predictions and true num_sold for five years')
    plt.show()

plot_five_years_combination(engineer)

# Residu model simpel

Setelah memahami plot nya, plot ini menunjukkan bahwa hari libur ada pada :
- Sekitar akhir tahun
- Tiga hari libur sesuai bulan penuh
- Setengah pertama bulan Mei, setengah kedua bulan Mei.
- Awal Juni dan Akhir Juni
- Awal November


In [ ]:
train_df['pred'] = np.exp(model.predict(preproc.transform(train_df[features])))
by_date = train_df.groupby(level='date')
residuals = (by_date.pred.sum() - by_date.num_sold.sum()) / (by_date.pred.sum() + by_date.num_sold.sum()) * 200

# Plot all residuals (four-year range, sum of all products)
def plot_all_residuals(residuals):
    plt.figure(figsize=(20,6))
    plt.scatter(residuals.index,
                residuals,
                s=1, color='k')
    plt.vlines(pd.date_range('2015-01-01', '2019-01-01', freq='M'),
               plt.ylim()[0], plt.ylim()[1], alpha=0.5)
    plt.vlines(pd.date_range('2015-01-01', '2019-01-01', freq='Y'),
               plt.ylim()[0], plt.ylim()[1], alpha=0.5)
    plt.title('Residuals for four years')
    plt.show()
    
plot_all_residuals(residuals)

# Plot residuals for interesting intervals
def plot_around(residuals, m, d, w):
    """Plot residuals in an interval of with 2*w around month=m and day=d"""
    plt.figure()
    plt.title(f"Residuals around m={m} d={d}")
    for y in np.arange(2015, 2020):
        d0 = pd.Timestamp(date(y, m, d))
        residual_range = residuals[(residuals.index > d0 - timedelta(w)) & 
                                   (residuals.index < d0 + timedelta(w))]
        plt.plot([(r - d0).days for r in residual_range.index], residual_range, label=str(y))
    plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True)) # only integer labels
    plt.legend()
    plt.show()

plot_around(residuals, 1, 1, 20) # end of year peak
plot_around(residuals, 5, 1, 50) # three moveable peaks depending on Easter
#plot_around(residuals, 5, 21, 10) # zoom-in
#plot_around(residuals, 5, 31, 15) # zoom-in
plot_around(residuals, 6, 10, 10) # first half of June (with overlay of Pentecost in 2017)
plot_around(residuals, 6, 30, 10) # moveable peak end of June
plot_around(residuals, 11, 5, 10) # moveable peak beginning of November

# Pembetulan hari libur

In [ ]:
# Feature engineering
def engineer_more(df):
    """Return a new dataframe with more engineered features"""
    new_df = engineer(df)

    # End of year
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d)
                                      for d in range(24, 32)})],
                       axis=1)
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) 
                                      for d in range(1, 13)})],
                       axis=1)
    
    # May
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) 
                                      for d in list(range(1, 10)) + list(range(17, 25))})],
                       axis=1)
    
    # June
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"june{d}":
                                      (df.date.dt.month == 6) & (df.date.dt.day == d) 
                                      for d in list(range(6, 14))})],
                       axis=1)
    
    # Last Wednesday of June
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"wed_june{d}": 
                                      (df.date - wed_june_date == np.timedelta64(d, "D"))
                                      for d in list(range(-5, 6))})],
                       axis=1)
    
    # First Sunday of November
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"sun_nov{d}": 
                                      (df.date - sun_nov_date == np.timedelta64(d, "D"))
                                      for d in list(range(0, 10))})],
                       axis=1)
    
    # Easter
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"easter{d}": 
                                      (df.date - easter_date == np.timedelta64(d, "D"))
                                      for d in list(range(0, 9)) + list(range(50, 60)) + list(range(40, 46))})],
                       axis=1)
    
    # Growth is country-specific
    #for country in ['Finland', 'Norway', 'Sweden']:
    #    new_df[f"{country}_year"] = (df.country == country) * (df.date.dt.year - 2016)
    #    new_df[f"{country}_peak_year"] = (df.country == country) * (new_df.dec29 | new_df.dec30 | new_df.easter_week) * (df.date.dt.year - 2016)
        
    return new_df.astype(np.float32)

train_df = engineer_more(original_train_df)
train_df['date'] = original_train_df.date
train_df['num_sold'] = original_train_df.num_sold.astype(np.float32)
test_df = engineer_more(original_test_df)
test_df.year = 2018 # no growth patch, see https://www.kaggle.com/c/tabular-playground-series-jan-2022/discussion/298318

features = test_df.columns
print(list(features))


# Melatih advanced model

Diagram menunjukkan bahwa residu sangat lebih kecil sehingga hari libur bisa dimasukkan dengan tepat.

In [ ]:
preproc, model = fit_model(train_df)
train_df['pred'] = np.exp(model.predict(preproc.transform(train_df[features])))
by_date = train_df.groupby(level='date')
residuals = (by_date.pred.sum() - by_date.num_sold.sum()) / (by_date.pred.sum() + by_date.num_sold.sum()) * 200

# Plot all num_sold_true and num_sold_pred (five years) for one country-store-product combination
plot_five_years_combination(engineer_more)

# Plot all residuals (four-year range, sum of all products)
plot_all_residuals(residuals)

# Plot residuals for interesting intervals
plot_around(residuals, 1, 1, 20) # end of year peak
plot_around(residuals, 5, 1, 50) # three moveable peaks depending on Easter
#plot_around(residuals, 5, 21, 10) # zoom-in
#plot_around(residuals, 5, 31, 15) # zoom-in
plot_around(residuals, 6, 10, 10) # first half of June (with overlay of Pentecost in 2017)
plot_around(residuals, 6, 30, 10) # moveable peak end of June
plot_around(residuals, 11, 5, 10) # moveable peak beginning of November


# Train Model dengan Validasi

Model dilatih dengan data tanggal 2015 hingga 2017 dan menggunakan 2018 sebagai validasi. Pada validasi, akan ditunjukkan :
- Seberapa lama waktu eksekusi dan SMAPE (Symmetric Mean Absolute Percentage Error)
- Scatterplot ytrue vs ypred (seharusnya semuanya mendekati garis diagonal)


In [ ]:
#%%time
RUNS = 1 # should be 1. increase the number of runs only if you want see how the result depends on the random seed
OUTLIERS = True
TRAIN_VAL_CUT = datetime(2018, 1, 1)
LOSS_CORRECTION = 1 # correction factor between Huber loss and SMAPE: 1.035 ( for linear regression with MSE use 1.038)

# Make the results reproducible
np.random.seed(202100)

total_start_time = datetime.now()
for run in range(RUNS):
    fold = 0
    train_idx = np.arange(len(train_df))[train_df.date < TRAIN_VAL_CUT]
    val_idx = np.arange(len(train_df))[train_df.date > TRAIN_VAL_CUT]
    print(f"Fold {run}.{fold}")
    X_tr = train_df.iloc[train_idx]
    X_va = train_df.iloc[val_idx]
    
    preproc, model = fit_model(X_tr, X_va)

# Inference and submission

In [ ]:
# Fit the model on the complete training data
train_idx = np.arange(len(train_df))
X_tr = train_df.iloc[train_idx]
preproc, model = fit_model(X_tr, None)

plot_five_years_combination(engineer_more) # Quick check for debugging

# Inference for test
test_pred_list = []
test_pred_list.append(np.exp(model.predict(preproc.transform(test_df[features]))) * LOSS_CORRECTION)

if len(test_pred_list) > 0:
    # Create the submission file
    sub = original_test_df[['row_id']].copy()
    sub['num_sold'] = sum(test_pred_list) / len(test_pred_list)
    sub.to_csv('submission.csv', index=False)

    # Plot the distribution of the test predictions
    plt.figure(figsize=(16,3))
    plt.hist(train_df['num_sold'], bins=np.linspace(0, 3000, 201),
             density=True, label='Training')
    plt.hist(sub['num_sold'], bins=np.linspace(0, 3000, 201),
             density=True, rwidth=0.5, label='Test predictions')
    plt.xlabel('num_sold')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()


In [ ]:
sub